In [30]:
import pickle
import numpy as np
import os
import pathlib

In [31]:
DATA_PATH = os.path.join(pathlib.Path.cwd().parent, 'fetch_data', 'processed_data.pkl')

with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import LinearSVR
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import GridSearchCV


In [33]:
# Separação Treinamento - Teste

X_train, X_test, y_train, y_test = train_test_split(data[0], data[1], test_size=0.2, random_state=42)

In [34]:
# Modelo de Random Forest

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
ypred = rf.predict(X_test)
RMSE_RF = np.sqrt(mean_squared_error(y_test, ypred))
error_percent_rf = 100 * (10**RMSE_RF - 1)

In [35]:
# Modelo de Extreme Gradient Boosting

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
xgb_model.fit(X_train, y_train)
ypred = xgb_model.predict(X_test)
RMSE_XGB = np.sqrt(mean_squared_error(y_test, ypred))
error_percent_xgb = 100 * (10**RMSE_XGB - 1)

/home/thomaschiari/dev/insper/4sem/ml/Machine-Learning-Ames/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [36]:
# Support Vector Regressor Linear

svr = LinearSVR(random_state=42)
svr.fit(X_train, y_train)
ypred = svr.predict(X_test)
RMSE_SVR = np.sqrt(mean_squared_error(y_test, ypred))
error_percent_svr = 100 * (10**RMSE_SVR - 1)

/home/thomaschiari/dev/insper/4sem/ml/Machine-Learning-Ames/env/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/thomaschiari/dev/insper/4sem/ml/Machine-Learning-Ames/env/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [37]:
# Category Boosting Regressor

cat = CatBoostRegressor(random_state=42, verbose=False)
cat.fit(X_train, y_train)
ypred = cat.predict(X_test)
RMSE_CAT = np.sqrt(mean_squared_error(y_test, ypred))
error_percent_cat = 100 * (10**RMSE_CAT - 1)

In [38]:
# Category Boosting Regressor com Grid Search

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'iterations': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
    'loss_function': ['RMSE'],
    
}

model = CatBoostRegressor(random_state=42, verbose=False)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)
grid.fit(X_train, y_train)

best_params = grid.best_params_
best_score = grid.best_score_
best_model = CatBoostRegressor(random_state=42, verbose=False, **best_params)

best_model.fit(X_train, y_train)
ypred = best_model.predict(X_test)
RMSE_CAT_GRID = np.sqrt(mean_squared_error(y_test, ypred))
error_percent_cat_grid = 100 * (10**RMSE_CAT_GRID - 1)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[CV 2/5] END depth=3, iterations=100, learning_rate=0.05, loss_function=RMSE;, score=-0.004 total time=   1.3s
[CV 1/5] END depth=3, iterations=100, learning_rate=0.01, loss_function=RMSE;, score=-0.012 total time=   2.1s
[CV 1/5] END depth=3, iterations=100, learning_rate=0.05, loss_function=RMSE;, score=-0.004 total time=   3.0s
[CV 3/5] END depth=3, iterations=100, learning_rate=0.01, loss_function=RMSE;, score=-0.013 total time=   3.2s
[CV 3/5] END depth=3, iterations=100, learning_rate=0.05, loss_function=RMSE;, score=-0.005 total time=   3.4s
[CV 2/5] END depth=3, iterations=100, learning_rate=0.01, loss_function=RMSE;, score=-0.013 total time=   3.8s
[CV 4/5] END depth=3, iterations=100, learning_rate=0.01, loss_function=RMSE;, score=-0.012 total time=   4.6s
[CV 5/5] END depth=3, iterations=100, learning_rate=0.05, loss_function=RMSE;, score=-0.004 total time=   2.5s
[CV 5/5] END depth=3, iterations=100, learning_rate=0.01, loss_function=RMSE;, score=-0.011 total time=   4.9s
[